In [ ]:
!pip install BayesLDM --quiet
!pip install --upgrade jupyter ipywidgets --quiet
!pip install networkx --quiet

# JustWalk Steps

In [ ]:
from pybehavior.models import DataSet
from pybehavior.tools import Loader, Preprocessor
import pandas as pd
import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

In [ ]:
data = pd.read_pickle('data/walk/data7.pickle')
data.tolist()


# JustWalk EMA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import BayesLDM.BayesLDM as BayesLDM
from pybehavior.tools import MCMC_Parser, BayesModelBuilder

data:pd.DataFrame = pd.read_pickle('data/ema/data11.pickle')
data = data.rename(columns={'steps': 'behavior', 'cue_to_action': 'cue'})
data = data.loc[data.rnum == 17].reset_index(drop=True)

# fig, ax = plt.subplots()
# ax.plot(data[['E10']])
# plt.show()

# data.columns
# ['rnum', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12.1.1',
#        'E12.1.2', 'E12.4', 'E12.5', 'E12.6', 'E12.7', 'day_index',
#        'day_of_week', 'steps', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'self_efficacy',
#        'perceived_barrier', 'environmental_context', 'context_of_walking',
#        'typicalness_of_context', 'self_management_skills', 'cue_to_action',
#        'environmental_context_weekly', 'social_support_friends']

In [ ]:
# ['rnum', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12.1.1',
#        'E12.1.2', 'E12.4', 'E12.5', 'E12.6', 'E12.7', 'day_index',
#        'day_of_week', 'steps', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'self_efficacy',
#        'perceived_barrier', 'environmental_context', 'context_of_walking',
#        'typicalness_of_context', 'self_management_skills', 'cue_to_action',
#        'environmental_context_weekly', 'social_support_friends']

data.name = "data"

builder = BayesModelBuilder('model')


# builder.add_variable('g_self_efficacy', mean=0, stdev=1)
# builder.add_variable('b_self_efficacy', mean=0, stdev=1)
# builder.add_variable('s_self_efficacy', 'exponential', exponent=0.1)

# builder.add_variable('self_efficacy[0]', mean='0', stdev='1')
# builder.add_variable_autoregressive('self_efficacy', 't', "normal", gain='g_self_efficacy', bias='b_self_efficacy', stdev='s_self_efficacy')

builder.add_variable_regression_edge([
    ('self_efficacy', 'self_efficacy'),
    ('perceived_barrier', 'perceived_barrier'),
    ('perceived_barrier', 'self_efficacy'),
    ('behavior', 'behavior'),
    ('cue', 'cue'),
    ('self_efficacy', 'behavior'),
    ('cue', 'behavior')
    ])

print(builder.get_full_model())

N = data.shape[0]

for window_size in range(10, N):
    actual_window_size = window_size + 1        # including the last missing data
    for start_point in range(0, N - actual_window_size):
        print("cropping {}:{}".format(start_point, actual_window_size + start_point + 1))
        current_data = data.iloc[start_point:(start_point + actual_window_size) + 1]
        current_data.loc[actual_window_size:actual_window_size] = None  # type: ignore

        builder.set_index('t', 0, actual_window_size)



data.loc[10:10]


In [ ]:

# for size in range(10, 65):
    
#     builder.set_index('t', 0, 64)


# data.loc[data.t==64, 'self_efficacy'] = None  # type: ignore
# data.loc[data.t==64, 'perceived_barrier'] = None  # type: ignore
# data.loc[data.t==64, 'behavior'] = None  # type: ignore
# data.loc[data.t==64, 'cue'] = None  # type: ignore

# model = BayesLDM.compile(builder.get_full_model(), obs=['self_efficacy', 'perceived_barrier', 'cue', 'behavior'], data=[data]) 
# samples = model.sample(b_post_process=True)


parser = MCMC_Parser(model)

parser.variable['self_efficacy'][64]

In [ ]:
# ['rnum', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12.1.1',
#        'E12.1.2', 'E12.4', 'E12.5', 'E12.6', 'E12.7', 'day_index',
#        'day_of_week', 'steps', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'self_efficacy',
#        'perceived_barrier', 'environmental_context', 'context_of_walking',
#        'typicalness_of_context', 'self_management_skills', 'cue_to_action',
#        'environmental_context_weekly', 'social_support_friends']

data.name = "data"

builder = BayesModelBuilder('model')

builder.set_index('t', 0, 64)

builder.add_variable_regression_edge([('self_efficacy', 'perceived_barrier')])

print(builder.get_full_model())

# data.loc[data.t==64, 'self_efficacy'] = None  # type: ignore

# model = BayesLDM.compile(builder.get_full_model(), obs=['self_efficacy'], data=[data]) 
# model.sample(b_post_process=True)

# parser = MCMC_Parser(model)

# parser.__dict__

In [ ]:
            


parser = MCMC_Parser(model)

parser.__dict__

In [ ]:
data

In [3]:
data = pd.read_pickle('data/ema/data11.pickle')
total = data.groupby('rnum').t.count().reset_index()
items = [x for x in data.columns.to_list() if x.startswith('E')]
data2 = data[items].isna()
non_zero = data2.sum(axis=1) < 10
data['non_zero'] = non_zero
stat = data[['rnum', 'non_zero']].groupby('rnum').sum().reset_index()
data = pd.merge(stat, total, on="rnum")
data['non_zero'] = data['non_zero'] / data.t
display(data[data.non_zero > 0.6].sort_values('non_zero', ascending=False).reset_index(drop=True))
print(data[data.non_zero > 0.6].shape)
included_participants = data[data.non_zero > 0.6].rnum.to_list()
included_participants


,rnum,non_zero,t
0,17,0.984615,65
1,9,0.954023,87
2,26,0.943182,88
3,11,0.930233,86
4,47,0.897436,39
5,31,0.891892,74
6,22,0.886364,88
7,18,0.850575,87
8,25,0.848101,79
9,8,0.831325,83


(21, 3)


[4, 5, 6, 8, 9, 11, 17, 18, 22, 25, 26, 27, 30, 31, 33, 34, 42, 43, 44, 47, 48]